In [ ]:
# DO NOT REMOVE!
import os

import pickle

import spacy
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import numpy as np

#!pip install datasets
#from datasets import load_dataset

import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

#!pip install pytorch_model_summary
#from pytorch_model_summary import summary

In [ ]:
# DO NOT REMOVE OR MODIFY
# Check if GPU is available and determine the device
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

print(f'The available device is {device}')

The available device is cpu


In [ ]:
# DO NOT REMOVE! (unless you work locally)
# mount drive: WE NEED IT FOR SAVING IMAGES! NECESSARY FOR GOOGLE COLAB!
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import pandas as pd
df = pd.read_parquet('/content/gdrive/MyDrive/dataset1.parquet')
pd.set_option('display.max_columns', None)
#df.head()
print(df.columns)
columns_df = pd.DataFrame(df.columns)
columns_df.to_csv('columns.csv', index=False, header=False)

Index(['subject_id', 'heart_rate', 'sbp', 'dbp', 'mbp', 'sbp_ni', 'dbp_ni',
       'mbp_ni', 'resp_rate', 'spo2',
       ...
       'liver', 'renal', 'respiration', 'shock_index', 'shock_index*age',
       'so2fio2ratio', 'sepsis3_6h', 'sepsis3_12h', 'sepsis3_24h',
       'sepsis3_48h'],
      dtype='object', length=144)


In [ ]:
import pandas as pd


unique_subjects = df['subject_id'].nunique()

if unique_subjects >= 2000:
    # Randomly sample 2000 unique subject_ids
    sampled_subjects = df['subject_id'].drop_duplicates().sample(n=2000, random_state=42)

    df = df[df['subject_id'].isin(sampled_subjects)]
else:
    print(f"Only {unique_subjects} unique patients available, which is less than 2000.")

# df_sampled now contains all rows for the 2000 unique patients


In [ ]:
num_unique_subjects = df['subject_id'].nunique()
print(f"Number of unique subject_id values: {num_unique_subjects}")


Number of unique subject_id values: 2000


In [ ]:
df.head()


subject_id  heart_rate    sbp   dbp   mbp  \
stay_id  charttime                                                        
30004242 2180-11-19 00:00:00    17220323        69.0  165.0  73.0  96.0   
         2180-11-19 01:00:00    17220323        59.0  107.0  50.0  94.0   
         2180-11-19 02:00:00    17220323        60.0   92.0  74.0  78.0   
         2180-11-19 03:00:00    17220323        46.0  132.0  47.0  74.5   
         2180-11-19 04:00:00    17220323        46.0  145.0  52.0  83.0   

                              sbp_ni  dbp_ni  mbp_ni  resp_rate  spo2  \
stay_id  charttime                                                      
30004242 2180-11-19 00:00:00   165.0    73.0    65.0       27.5  98.0   
         2180-11-19 01:00:00   107.0    46.0    94.0       21.0  97.0   
         2180-11-19 02:00:00    92.0    74.0    78.0       19.0  97.0   
         2180-11-19 03:00:00   120.0    34.0    56.0       16.0  96.0   
         2180-11-19 04:00:00   120.0    34.0    56.0       18.0  95.0   

                                       admittime  anchor_age  anchor_year  \
stay_id  charttime                                                          
30004242 2180-11-19 00:00:00 2180-11-19 01:06:00          75         2180   
         2180-11-19 01:00:00 2180-11-19 01:06:00          75         2180   
         2180-11-19 02:00:00 2180-11-19 01:06:00          75         2180   
         2180-11-19 03:00:00 2180-11-19 01:06:00          75         2180   
         2180-11-19 04:00:00 2180-11-19 01:06:00          75         2180   

                              age  height  gender         dod  \
stay_id  charttime                                              
30004242 2180-11-19 00:00:00   75   163.0       1  2152-10-30   
         2180-11-19 01:00:00   75   163.0       1  2152-10-30   
         2180-11-19 02:00:00   75   163.0       1  2152-10-30   
         2180-11-19 03:00:00   75   163.0       1  2152-10-30   
         2180-11-19 04:00:00   75   163.0       1  2152-10-30   

                                       dischtime  los_hospital  admission_age  \
stay_id  charttime                                                              
30004242 2180-11-19 00:00:00 2180-11-26 15:00:00      7.579167             75   
         2180-11-19 01:00:00 2180-11-26 15:00:00      7.579167             75   
         2180-11-19 02:00:00 2180-11-26 15:00:00      7.579167             75   
         2180-11-19 03:00:00 2180-11-26 15:00:00      7.579167             75   
         2180-11-19 04:00:00 2180-11-26 15:00:00      7.579167             75   

                                                race  hospital_expire_flag  \
stay_id  charttime                                                           
30004242 2180-11-19 00:00:00  BLACK/AFRICAN AMERICAN                     0   
         2180-11-19 01:00:00  BLACK/AFRICAN AMERICAN                     0   
         2180-11-19 02:00:00  BLACK/AFRICAN AMERICAN                     0   
         2180-11-19 03:00:00  BLACK/AFRICAN AMERICAN                     0   
         2180-11-19 04:00:00  BLACK/AFRICAN AMERICAN                     0   

                              hospstay_seq  first_hosp_stay  \
stay_id  charttime                                            
30004242 2180-11-19 00:00:00             1             True   
         2180-11-19 01:00:00             1             True   
         2180-11-19 02:00:00             1             True   
         2180-11-19 03:00:00             1             True   
         2180-11-19 04:00:00             1             True   

                                      icu_intime         icu_outtime  los_icu  \
stay_id  charttime                                                              
30004242 2180-11-19 00:00:00 2180-11-19 01:06:36 2180-11-20 20:53:44     1.82   
         2180-11-19 01:00:00 2180-11-19 01:06:36 2180-11-20 20:53:44     1.82   
         2180-11-19 02:00:00 2180-11-19 01:06:36 2180-11-20 20:53:44     1.82   
         2180-11-19 03:00:00 2180-

In [ ]:
#DROP
df = df.drop(columns=['sepsis3_6h', 'sepsis3_12h', 'sepsis3_24h', 'sepsis3_48h'])
df.head()

subject_id  heart_rate    sbp   dbp   mbp  \
stay_id  charttime                                                        
30004242 2180-11-19 00:00:00    17220323        69.0  165.0  73.0  96.0   
         2180-11-19 01:00:00    17220323        59.0  107.0  50.0  94.0   
         2180-11-19 02:00:00    17220323        60.0   92.0  74.0  78.0   
         2180-11-19 03:00:00    17220323        46.0  132.0  47.0  74.5   
         2180-11-19 04:00:00    17220323        46.0  145.0  52.0  83.0   

                              sbp_ni  dbp_ni  mbp_ni  resp_rate  spo2  \
stay_id  charttime                                                      
30004242 2180-11-19 00:00:00   165.0    73.0    65.0       27.5  98.0   
         2180-11-19 01:00:00   107.0    46.0    94.0       21.0  97.0   
         2180-11-19 02:00:00    92.0    74.0    78.0       19.0  97.0   
         2180-11-19 03:00:00   120.0    34.0    56.0       16.0  96.0   
         2180-11-19 04:00:00   120.0    34.0    56.0       18.0  95.0   

                                       admittime  anchor_age  anchor_year  \
stay_id  charttime                                                          
30004242 2180-11-19 00:00:00 2180-11-19 01:06:00          75         2180   
         2180-11-19 01:00:00 2180-11-19 01:06:00          75         2180   
         2180-11-19 02:00:00 2180-11-19 01:06:00          75         2180   
         2180-11-19 03:00:00 2180-11-19 01:06:00          75         2180   
         2180-11-19 04:00:00 2180-11-19 01:06:00          75         2180   

                              age  height  gender         dod  \
stay_id  charttime                                              
30004242 2180-11-19 00:00:00   75   163.0       1  2152-10-30   
         2180-11-19 01:00:00   75   163.0       1  2152-10-30   
         2180-11-19 02:00:00   75   163.0       1  2152-10-30   
         2180-11-19 03:00:00   75   163.0       1  2152-10-30   
         2180-11-19 04:00:00   75   163.0       1  2152-10-30   

                                       dischtime  los_hospital  admission_age  \
stay_id  charttime                                                              
30004242 2180-11-19 00:00:00 2180-11-26 15:00:00      7.579167             75   
         2180-11-19 01:00:00 2180-11-26 15:00:00      7.579167             75   
         2180-11-19 02:00:00 2180-11-26 15:00:00      7.579167             75   
         2180-11-19 03:00:00 2180-11-26 15:00:00      7.579167             75   
         2180-11-19 04:00:00 2180-11-26 15:00:00      7.579167             75   

                                                race  hospital_expire_flag  \
stay_id  charttime                                                           
30004242 2180-11-19 00:00:00  BLACK/AFRICAN AMERICAN                     0   
         2180-11-19 01:00:00  BLACK/AFRICAN AMERICAN                     0   
         2180-11-19 02:00:00  BLACK/AFRICAN AMERICAN                     0   
         2180-11-19 03:00:00  BLACK/AFRICAN AMERICAN                     0   
         2180-11-19 04:00:00  BLACK/AFRICAN AMERICAN                     0   

                              hospstay_seq  first_hosp_stay  \
stay_id  charttime                                            
30004242 2180-11-19 00:00:00             1             True   
         2180-11-19 01:00:00             1             True   
         2180-11-19 02:00:00             1             True   
         2180-11-19 03:00:00             1             True   
         2180-11-19 04:00:00             1             True   

                                      icu_intime         icu_outtime  los_icu  \
stay_id  charttime                                                              
30004242 2180-11-19 00:00:00 2180-11-19 01:06:36 2180-11-20 20:53:44     1.82   
         2180-11-19 01:00:00 2180-11-19 01:06:36 2180-11-20 20:53:44     1.82   
         2180-11-19 02:00:00 2180-11-19 01:06:36 2180-11-20 20:53:44     1.82   
         2180-11-19 03:00:00 2180-

In [ ]:
df['admittime'] = pd.to_datetime(df['admittime'])
df['dischtime'] = pd.to_datetime(df['dischtime'])

df['hospital_stay_duration'] = df['dischtime'] - df['admittime']

df = df.drop(columns=['admittime', 'dischtime'])
cols = df.columns.tolist()

cols = [cols.pop()] + cols

df = df[cols]


df['icu_intime'] = pd.to_datetime(df['icu_intime'])
df['icu_outtime'] = pd.to_datetime(df['icu_outtime'])

df['icu_stay_duration'] = df['icu_outtime'] - df['icu_intime']

df = df.drop(columns=['icu_intime', 'icu_outtime'])

icu_stay_duration = df.pop('icu_stay_duration')
df.insert(0, 'icu_stay_duration', icu_stay_duration)





In [ ]:
df = df.reset_index()
# Display the updated DataFrame
df.head()


,stay_id,charttime,icu_stay_duration,hospital_stay_duration,subject_id,heart_rate,sbp,dbp,mbp,sbp_ni,dbp_ni,mbp_ni,resp_rate,spo2,anchor_age,anchor_year,age,height,gender,dod,los_hospital,admission_age,race,hospital_expire_flag,hospstay_seq,first_hosp_stay,los_icu,icustay_seq,first_icu_stay,weight_admit,weight_daily,so2,po2,pco2,fio2_chartevents,fio2,aado2,aado2_calc,pao2fio2ratio,ph,baseexcess,totalco2,carboxyhemoglobin,methemoglobin,lactate,temperature,d_dimer,fibrinogen,thrombin,inr,pt,ptt,albumin,globulin,total_protein,aniongap,bun,creatinine,bicarbonate,calcium,chloride,glucose,potassium,sodium,mch,mchc,mcv,platelet,rbc,rdw,wbc,hematocrit,hemoglobin,urineoutput,gcs,gcs_motor,gcs_verbal,gcs_eyes,gcs_unable,alt,alp,ast,amylase,bilirubin_total,bilirubin_direct,bilirubin_indirect,ck_cpk,ck_mb,ggt,ld_ldh,vaso_rate_dobutamine,vaso_amount_dobutamine,vaso_rate_dopamine,vaso_amount_dopamine,vaso_rate_epinephrine,vaso_amount_epinephrine,vaso_rate_norepinephrine,vaso_amount_norepinephrine,vaso_rate_vasopressin,vaso_amount_vasopressin,dopamine,epinephrine,norepinephrine,phenylephrine,vasopressin,dobutamine,milrinone,pao2fio2ratio_novent,pao2fio2ratio_vent,rate_epinephrine,rate_norepinephrine,rate_dopamine,rate_dobutamine,meanbp_min,gcs_min,uo_24hr,bilirubin_max,creatinine_max,platelet_min,respiration_24hours,coagulation_24hours,liver_24hours,cardiovascular_24hours,cns_24hours,renal_24hours,sofa_24hours,antibiotic,positive_culture,suspected_infection,sepsis3,sofa_score,cardiovascular,cns,coagulation,liver,renal,respiration,shock_index,shock_index*age,so2fio2ratio
0,30004242,2180-11-19 00:00:00,1 days 19:47:08,7 days 13:54:00,17220323,69.0,165.0,73.0,96.0,165.0,73.0,65.0,27.5,98.0,75,2180,75,163.0,1,2152-10-30,7.579167,75,BLACK/AFRICAN AMERICAN,0,1,True,1.82,1,True,76.2,80.1,97.0,149.0,38.0,40.0,50.0,172.0,88.7,372.5,7.37,-2.0,23.0,0.0,0.0,1.4,36.78,9551.0,254.0,23.4,1.2,12.9,34.5,4.4,3.4,6.9,15.0,24.0,0.8,23.0,8.2,107.0,113.0,3.7,141.0,31.7,33.7,94.0,161.0,3.06,13.5,5.6,28.8,9.7,50.0,15.0,6.0,5.0,4.0,0.0,31.0,572.0,29.0,146.0,1.2,4.2,1.3,334.0,9.0,25.0,188.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.500884,0.01,0.030002,1.997283,2.4,0.500057,0.271739,268.571429,372.5,0.01,0.030002,2.500884,0.500057,96.0,15.0,1183.742591,1.2,0.8,161.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,3.0,1.0,2.0,0.0,0.0,0.0,0.0,0.418182,31.363636,1.94
1,30004242,2180-11-19 01:00:00,1 days 19:47:08,7 days 13:54:00,17220323,59.0,107.0,50.0,94.0,107.0,46.0,94.0,21.0,97.0,75,2180,75,163.0,1,2152-10-30,7.579167,75,BLACK/AFRICAN AMERICAN,0,1,True,1.82,1,True,76.2,80.1,97.0,149.0,38.0,40.0,50.0,172.0,88.7,372.5,7.37,-2.0,23.0,0.0,0.0,1.4,36.72,9551.0,254.0,23.4,1.2,12.9,34.5,4.4,3.4,6.9,15.0,24.0,0.8,23.0,8.2,107.0,113.0,3.7,141.0,31.7,33.7,94.0,161.0,3.06,13.5,5.6,28.8,9.7,50.0,15.0,6.0,5.0,4.0,0.0,31.0,572.0,29.0,146.0,1.2,4.2,1.3,334.0,9.0,25.0,188.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.500884,0.01,0.030002,1.997283,2.4,0.500057,0.271739,268.571429,372.5,0.01,0.030002,2.500884,0.500057,78.0,15.0,1183.742591,1.2,0.8,161.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,3.0,0.5,1.0,0.0,0.0,0.0,0.0,0.551402,41.355140,1.94
2,30004242,2180-11-19 02:00:00,1 days 19:47:08,7 days 13:54:00,17220323,60.0,92.0,74.0,78.0,92.0,74.0,78.0,19.0,97.0,75,2180,75,163.0,1,2152-10-30,7.579167,75,BLACK/AFRICAN AMERICAN,0,1,True,1.82,1,True,76.2,80.1,97.0,149.0,38.0,40.0,50.0,172.0,88.7,372.5,7.37,-2.0,23.0,0.0,0.0,1.4,36.72,9551.0,254.0,23.4,1.2,12.9,34.5,4.4,3.4,6.9,15.0,24.0,0.8,23.0,8.2,107.0,113.0,3.7,141.0,31.7,33.7,94.0,161.0,3.06,13.5,5.6,28.8,9.7,50.0,15.0,6.0,5.0,4.0,0.0,31.0,572.0,29.0,146.0,1.2,4.2,1.3,334.0,9.0,25.0,188.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.500884,0.01,0.030002,1.997283,2.4,0.500057,0.271739,268.571429,372.5,0.01,0.030002,2.500884,0.500057,56.0,15.0,1183.742591,1.2,0.8,161.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0,0.0,0.0,0,3.0,1.0,1.0,0.0,0.0,0.0,0.0,0.652174,48.913043,1.94
3,30004242,2180-11-19 03:00:00,1 days 19:47:08,7 days 13:54:00,17220323,46.0,132.0,47.0,74.5

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

label_encoder = LabelEncoder()

# Verify the column names in your DataFrame
print(df.columns)

df['gender_encoded'] = label_encoder.fit_transform(df['gender'])  # Make sure 'gender' exists
df['first_hosp_stay_encoded'] = label_encoder.fit_transform(df['first_hosp_stay'])

df['first_icu_stay_encoded'] = label_encoder.fit_transform(df['first_icu_stay'])
df['race_encoded'] = label_encoder.fit_transform(df['race'])
df.head()

Index(['stay_id', 'charttime', 'icu_stay_duration', 'hospital_stay_duration',
       'subject_id', 'heart_rate', 'sbp', 'dbp', 'mbp', 'sbp_ni',
       ...
       'sofa_score', 'cardiovascular', 'cns', 'coagulation', 'liver', 'renal',
       'respiration', 'shock_index', 'shock_index*age', 'so2fio2ratio'],
      dtype='object', length=140)


,stay_id,charttime,icu_stay_duration,hospital_stay_duration,subject_id,heart_rate,sbp,dbp,mbp,sbp_ni,dbp_ni,mbp_ni,resp_rate,spo2,anchor_age,anchor_year,age,height,gender,dod,los_hospital,admission_age,race,hospital_expire_flag,hospstay_seq,first_hosp_stay,los_icu,icustay_seq,first_icu_stay,weight_admit,weight_daily,so2,po2,pco2,fio2_chartevents,fio2,aado2,aado2_calc,pao2fio2ratio,ph,baseexcess,totalco2,carboxyhemoglobin,methemoglobin,lactate,temperature,d_dimer,fibrinogen,thrombin,inr,pt,ptt,albumin,globulin,total_protein,aniongap,bun,creatinine,bicarbonate,calcium,chloride,glucose,potassium,sodium,mch,mchc,mcv,platelet,rbc,rdw,wbc,hematocrit,hemoglobin,urineoutput,gcs,gcs_motor,gcs_verbal,gcs_eyes,gcs_unable,alt,alp,ast,amylase,bilirubin_total,bilirubin_direct,bilirubin_indirect,ck_cpk,ck_mb,ggt,ld_ldh,vaso_rate_dobutamine,vaso_amount_dobutamine,vaso_rate_dopamine,vaso_amount_dopamine,vaso_rate_epinephrine,vaso_amount_epinephrine,vaso_rate_norepinephrine,vaso_amount_norepinephrine,vaso_rate_vasopressin,vaso_amount_vasopressin,dopamine,epinephrine,norepinephrine,phenylephrine,vasopressin,dobutamine,milrinone,pao2fio2ratio_novent,pao2fio2ratio_vent,rate_epinephrine,rate_norepinephrine,rate_dopamine,rate_dobutamine,meanbp_min,gcs_min,uo_24hr,bilirubin_max,creatinine_max,platelet_min,respiration_24hours,coagulation_24hours,liver_24hours,cardiovascular_24hours,cns_24hours,renal_24hours,sofa_24hours,antibiotic,positive_culture,suspected_infection,sepsis3,sofa_score,cardiovascular,cns,coagulation,liver,renal,respiration,shock_index,shock_index*age,so2fio2ratio,gender_encoded,first_hosp_stay_encoded,first_icu_stay_encoded,race_encoded
0,30004242,2180-11-19 00:00:00,1 days 19:47:08,7 days 13:54:00,17220323,69.0,165.0,73.0,96.0,165.0,73.0,65.0,27.5,98.0,75,2180,75,163.0,1,2152-10-30,7.579167,75,BLACK/AFRICAN AMERICAN,0,1,True,1.82,1,True,76.2,80.1,97.0,149.0,38.0,40.0,50.0,172.0,88.7,372.5,7.37,-2.0,23.0,0.0,0.0,1.4,36.78,9551.0,254.0,23.4,1.2,12.9,34.5,4.4,3.4,6.9,15.0,24.0,0.8,23.0,8.2,107.0,113.0,3.7,141.0,31.7,33.7,94.0,161.0,3.06,13.5,5.6,28.8,9.7,50.0,15.0,6.0,5.0,4.0,0.0,31.0,572.0,29.0,146.0,1.2,4.2,1.3,334.0,9.0,25.0,188.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.500884,0.01,0.030002,1.997283,2.4,0.500057,0.271739,268.571429,372.5,0.01,0.030002,2.500884,0.500057,96.0,15.0,1183.742591,1.2,0.8,161.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,3.0,1.0,2.0,0.0,0.0,0.0,0.0,0.418182,31.363636,1.94,1,1,1,7
1,30004242,2180-11-19 01:00:00,1 days 19:47:08,7 days 13:54:00,17220323,59.0,107.0,50.0,94.0,107.0,46.0,94.0,21.0,97.0,75,2180,75,163.0,1,2152-10-30,7.579167,75,BLACK/AFRICAN AMERICAN,0,1,True,1.82,1,True,76.2,80.1,97.0,149.0,38.0,40.0,50.0,172.0,88.7,372.5,7.37,-2.0,23.0,0.0,0.0,1.4,36.72,9551.0,254.0,23.4,1.2,12.9,34.5,4.4,3.4,6.9,15.0,24.0,0.8,23.0,8.2,107.0,113.0,3.7,141.0,31.7,33.7,94.0,161.0,3.06,13.5,5.6,28.8,9.7,50.0,15.0,6.0,5.0,4.0,0.0,31.0,572.0,29.0,146.0,1.2,4.2,1.3,334.0,9.0,25.0,188.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.500884,0.01,0.030002,1.997283,2.4,0.500057,0.271739,268.571429,372.5,0.01,0.030002,2.500884,0.500057,78.0,15.0,1183.742591,1.2,0.8,161.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,3.0,0.5,1.0,0.0,0.0,0.0,0.0,0.551402,41.355140,1.94,1,1,1,7
2,30004242,2180-11-19 02:00:00,1 days 19:47:08,7 days 13:54:00,17220323,60.0,92.0,74.0,78.0,92.0,74.0,78.0,19.0,97.0,75,2180,75,163.0,1,2152-10-30,7.579167,75,BLACK/AFRICAN AMERICAN,0,1,True,1.82,1,True,76.2,80.1,97.0,149.0,38.0,40.0,50.0,172.0,88.7,372.5,7.37,-2.0,23.0,0.0,0.0,1.4,36.72,9551.0,254.0,23.4,1.2,12.9,34.5,4.4,3.4,6.9,15.0,24.0,0.8,23.0,8.2,107.0,113.0,3.7,141.0,31.7,33.7,94.0,161.0,3.06,13.5,5.6,28.8,9.7,50.0,15.0,6.0,5.0,4.0,0.0,31.0,572.0,29.0,146.0,1.2,4.2,1.3,334.0,9.0,25.0,188.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.500884,0.01,0.030002,1.997283,2.4,0.500057,0.271739,268.571429,372.5,0.01,0.030002,2.500884,0.500057,56.0,15.0,1183.742591,1.2,0.8,161.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0,0.0,0.0,0,3.0,1.0,1.0,0.0,0.0,0.0,0.0,0.652174,48.913043,1.94,1

In [ ]:
# Dropping multiple columns
df = df.drop(columns=['gender', 'race', 'first_icu_stay', 'first_hosp_stay','subject_id'])

# Display the DataFrame to verify the changes
df.head()

,stay_id,charttime,icu_stay_duration,hospital_stay_duration,heart_rate,sbp,dbp,mbp,sbp_ni,dbp_ni,mbp_ni,resp_rate,spo2,anchor_age,anchor_year,age,height,dod,los_hospital,admission_age,hospital_expire_flag,hospstay_seq,los_icu,icustay_seq,weight_admit,weight_daily,so2,po2,pco2,fio2_chartevents,fio2,aado2,aado2_calc,pao2fio2ratio,ph,baseexcess,totalco2,carboxyhemoglobin,methemoglobin,lactate,temperature,d_dimer,fibrinogen,thrombin,inr,pt,ptt,albumin,globulin,total_protein,aniongap,bun,creatinine,bicarbonate,calcium,chloride,glucose,potassium,sodium,mch,mchc,mcv,platelet,rbc,rdw,wbc,hematocrit,hemoglobin,urineoutput,gcs,gcs_motor,gcs_verbal,gcs_eyes,gcs_unable,alt,alp,ast,amylase,bilirubin_total,bilirubin_direct,bilirubin_indirect,ck_cpk,ck_mb,ggt,ld_ldh,vaso_rate_dobutamine,vaso_amount_dobutamine,vaso_rate_dopamine,vaso_amount_dopamine,vaso_rate_epinephrine,vaso_amount_epinephrine,vaso_rate_norepinephrine,vaso_amount_norepinephrine,vaso_rate_vasopressin,vaso_amount_vasopressin,dopamine,epinephrine,norepinephrine,phenylephrine,vasopressin,dobutamine,milrinone,pao2fio2ratio_novent,pao2fio2ratio_vent,rate_epinephrine,rate_norepinephrine,rate_dopamine,rate_dobutamine,meanbp_min,gcs_min,uo_24hr,bilirubin_max,creatinine_max,platelet_min,respiration_24hours,coagulation_24hours,liver_24hours,cardiovascular_24hours,cns_24hours,renal_24hours,sofa_24hours,antibiotic,positive_culture,suspected_infection,sepsis3,sofa_score,cardiovascular,cns,coagulation,liver,renal,respiration,shock_index,shock_index*age,so2fio2ratio,gender_encoded,first_hosp_stay_encoded,first_icu_stay_encoded,race_encoded
0,30004242,2180-11-19 00:00:00,1 days 19:47:08,7 days 13:54:00,69.0,165.0,73.0,96.0,165.0,73.0,65.0,27.5,98.0,75,2180,75,163.0,2152-10-30,7.579167,75,0,1,1.82,1,76.2,80.1,97.0,149.0,38.0,40.0,50.0,172.0,88.7,372.5,7.37,-2.0,23.0,0.0,0.0,1.4,36.78,9551.0,254.0,23.4,1.2,12.9,34.5,4.4,3.4,6.9,15.0,24.0,0.8,23.0,8.2,107.0,113.0,3.7,141.0,31.7,33.7,94.0,161.0,3.06,13.5,5.6,28.8,9.7,50.0,15.0,6.0,5.0,4.0,0.0,31.0,572.0,29.0,146.0,1.2,4.2,1.3,334.0,9.0,25.0,188.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.500884,0.01,0.030002,1.997283,2.4,0.500057,0.271739,268.571429,372.5,0.01,0.030002,2.500884,0.500057,96.0,15.0,1183.742591,1.2,0.8,161.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,3.0,1.0,2.0,0.0,0.0,0.0,0.0,0.418182,31.363636,1.94,1,1,1,7
1,30004242,2180-11-19 01:00:00,1 days 19:47:08,7 days 13:54:00,59.0,107.0,50.0,94.0,107.0,46.0,94.0,21.0,97.0,75,2180,75,163.0,2152-10-30,7.579167,75,0,1,1.82,1,76.2,80.1,97.0,149.0,38.0,40.0,50.0,172.0,88.7,372.5,7.37,-2.0,23.0,0.0,0.0,1.4,36.72,9551.0,254.0,23.4,1.2,12.9,34.5,4.4,3.4,6.9,15.0,24.0,0.8,23.0,8.2,107.0,113.0,3.7,141.0,31.7,33.7,94.0,161.0,3.06,13.5,5.6,28.8,9.7,50.0,15.0,6.0,5.0,4.0,0.0,31.0,572.0,29.0,146.0,1.2,4.2,1.3,334.0,9.0,25.0,188.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.500884,0.01,0.030002,1.997283,2.4,0.500057,0.271739,268.571429,372.5,0.01,0.030002,2.500884,0.500057,78.0,15.0,1183.742591,1.2,0.8,161.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,3.0,0.5,1.0,0.0,0.0,0.0,0.0,0.551402,41.355140,1.94,1,1,1,7
2,30004242,2180-11-19 02:00:00,1 days 19:47:08,7 days 13:54:00,60.0,92.0,74.0,78.0,92.0,74.0,78.0,19.0,97.0,75,2180,75,163.0,2152-10-30,7.579167,75,0,1,1.82,1,76.2,80.1,97.0,149.0,38.0,40.0,50.0,172.0,88.7,372.5,7.37,-2.0,23.0,0.0,0.0,1.4,36.72,9551.0,254.0,23.4,1.2,12.9,34.5,4.4,3.4,6.9,15.0,24.0,0.8,23.0,8.2,107.0,113.0,3.7,141.0,31.7,33.7,94.0,161.0,3.06,13.5,5.6,28.8,9.7,50.0,15.0,6.0,5.0,4.0,0.0,31.0,572.0,29.0,146.0,1.2,4.2,1.3,334.0,9.0,25.0,188.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.500884,0.01,0.030002,1.997283,2.4,0.500057,0.271739,268.571429,372.5,0.01,0.030002,2.500884,0.500057,56.0,15.0,1183.742591,1.2,0.8,161.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0,0.0,0.0,0,3.0,1.0,1.0,0.0,0.0,0.0,0.0,0.652174,48.913043,1.94,1,1,1,7
3,30004242,2180-11-19 03:00:00,1 days 19:47:08,7 days 13:54:00,46.0,132.0,47.0,74.5,120.0,34.0,56.0,16.0,96.0,75,2180,75,163.0,2152-10-30,7.579167,75,0,1,1.82,1,76.2,80.1,97.0,149

In [ ]:
columns = df.columns.tolist()

last_four_columns = columns[-4:]

remaining_columns = columns[:-4]

df = df[last_four_columns + remaining_columns]

columns = df.columns.tolist()

fifth_column = columns[4]

remaining_columns = columns[:4] + columns[5:]

df = df[[fifth_column] + remaining_columns]

df.head()
#df.to_csv('dataframe.csv', index=False)


,stay_id,gender_encoded,first_hosp_stay_encoded,first_icu_stay_encoded,race_encoded,charttime,icu_stay_duration,hospital_stay_duration,heart_rate,sbp,dbp,mbp,sbp_ni,dbp_ni,mbp_ni,resp_rate,spo2,anchor_age,anchor_year,age,height,dod,los_hospital,admission_age,hospital_expire_flag,hospstay_seq,los_icu,icustay_seq,weight_admit,weight_daily,so2,po2,pco2,fio2_chartevents,fio2,aado2,aado2_calc,pao2fio2ratio,ph,baseexcess,totalco2,carboxyhemoglobin,methemoglobin,lactate,temperature,d_dimer,fibrinogen,thrombin,inr,pt,ptt,albumin,globulin,total_protein,aniongap,bun,creatinine,bicarbonate,calcium,chloride,glucose,potassium,sodium,mch,mchc,mcv,platelet,rbc,rdw,wbc,hematocrit,hemoglobin,urineoutput,gcs,gcs_motor,gcs_verbal,gcs_eyes,gcs_unable,alt,alp,ast,amylase,bilirubin_total,bilirubin_direct,bilirubin_indirect,ck_cpk,ck_mb,ggt,ld_ldh,vaso_rate_dobutamine,vaso_amount_dobutamine,vaso_rate_dopamine,vaso_amount_dopamine,vaso_rate_epinephrine,vaso_amount_epinephrine,vaso_rate_norepinephrine,vaso_amount_norepinephrine,vaso_rate_vasopressin,vaso_amount_vasopressin,dopamine,epinephrine,norepinephrine,phenylephrine,vasopressin,dobutamine,milrinone,pao2fio2ratio_novent,pao2fio2ratio_vent,rate_epinephrine,rate_norepinephrine,rate_dopamine,rate_dobutamine,meanbp_min,gcs_min,uo_24hr,bilirubin_max,creatinine_max,platelet_min,respiration_24hours,coagulation_24hours,liver_24hours,cardiovascular_24hours,cns_24hours,renal_24hours,sofa_24hours,antibiotic,positive_culture,suspected_infection,sepsis3,sofa_score,cardiovascular,cns,coagulation,liver,renal,respiration,shock_index,shock_index*age,so2fio2ratio
0,30004242,1,1,1,7,2180-11-19 00:00:00,1 days 19:47:08,7 days 13:54:00,69.0,165.0,73.0,96.0,165.0,73.0,65.0,27.5,98.0,75,2180,75,163.0,2152-10-30,7.579167,75,0,1,1.82,1,76.2,80.1,97.0,149.0,38.0,40.0,50.0,172.0,88.7,372.5,7.37,-2.0,23.0,0.0,0.0,1.4,36.78,9551.0,254.0,23.4,1.2,12.9,34.5,4.4,3.4,6.9,15.0,24.0,0.8,23.0,8.2,107.0,113.0,3.7,141.0,31.7,33.7,94.0,161.0,3.06,13.5,5.6,28.8,9.7,50.0,15.0,6.0,5.0,4.0,0.0,31.0,572.0,29.0,146.0,1.2,4.2,1.3,334.0,9.0,25.0,188.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.500884,0.01,0.030002,1.997283,2.4,0.500057,0.271739,268.571429,372.5,0.01,0.030002,2.500884,0.500057,96.0,15.0,1183.742591,1.2,0.8,161.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,3.0,1.0,2.0,0.0,0.0,0.0,0.0,0.418182,31.363636,1.94
1,30004242,1,1,1,7,2180-11-19 01:00:00,1 days 19:47:08,7 days 13:54:00,59.0,107.0,50.0,94.0,107.0,46.0,94.0,21.0,97.0,75,2180,75,163.0,2152-10-30,7.579167,75,0,1,1.82,1,76.2,80.1,97.0,149.0,38.0,40.0,50.0,172.0,88.7,372.5,7.37,-2.0,23.0,0.0,0.0,1.4,36.72,9551.0,254.0,23.4,1.2,12.9,34.5,4.4,3.4,6.9,15.0,24.0,0.8,23.0,8.2,107.0,113.0,3.7,141.0,31.7,33.7,94.0,161.0,3.06,13.5,5.6,28.8,9.7,50.0,15.0,6.0,5.0,4.0,0.0,31.0,572.0,29.0,146.0,1.2,4.2,1.3,334.0,9.0,25.0,188.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.500884,0.01,0.030002,1.997283,2.4,0.500057,0.271739,268.571429,372.5,0.01,0.030002,2.500884,0.500057,78.0,15.0,1183.742591,1.2,0.8,161.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,3.0,0.5,1.0,0.0,0.0,0.0,0.0,0.551402,41.355140,1.94
2,30004242,1,1,1,7,2180-11-19 02:00:00,1 days 19:47:08,7 days 13:54:00,60.0,92.0,74.0,78.0,92.0,74.0,78.0,19.0,97.0,75,2180,75,163.0,2152-10-30,7.579167,75,0,1,1.82,1,76.2,80.1,97.0,149.0,38.0,40.0,50.0,172.0,88.7,372.5,7.37,-2.0,23.0,0.0,0.0,1.4,36.72,9551.0,254.0,23.4,1.2,12.9,34.5,4.4,3.4,6.9,15.0,24.0,0.8,23.0,8.2,107.0,113.0,3.7,141.0,31.7,33.7,94.0,161.0,3.06,13.5,5.6,28.8,9.7,50.0,15.0,6.0,5.0,4.0,0.0,31.0,572.0,29.0,146.0,1.2,4.2,1.3,334.0,9.0,25.0,188.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.500884,0.01,0.030002,1.997283,2.4,0.500057,0.271739,268.571429,372.5,0.01,0.030002,2.500884,0.500057,56.0,15.0,1183.742591,1.2,0.8,161.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0,0.0,0.0,0,3.0,1.0,1.0,0.0,0.0,0.0,0.0,0.652174,48.913043,1.94
3,30004242,1,1,1,7,2180-11-19 03:00:00,1 days 19:47:08,7 days 13:54:00,46.0,132.0,47.0,74.5,120.0,34.0,56.0,16.0,96.0,75,2180,75,163.0,2152-10-30,7.579167,75,0,1,1.82,1,76.2,80.1,

In [ ]:
import pandas as pd

#df = df.drop(columns=['index'])

# Convert Timedelta to total seconds
if 'icu_stay_duration' in df.columns:
    # Convert Timedelta to total seconds
    df['icu_stay_duration'] = df['icu_stay_duration'].dt.total_seconds()
else:
    print("Warning: 'icu_stay_duration' column not found in the DataFrame.")

if 'hospital_stay_duration' in df.columns:
    # Convert Timedelta to total seconds
    df['hospital_stay_duration'] = df['hospital_stay_duration'].dt.total_seconds()
else:
    print("Warning: 'hospital_stay_duration' column not found in the DataFrame.")



In [ ]:
df.head()

,stay_id,gender_encoded,first_hosp_stay_encoded,first_icu_stay_encoded,race_encoded,charttime,icu_stay_duration,hospital_stay_duration,heart_rate,sbp,dbp,mbp,sbp_ni,dbp_ni,mbp_ni,resp_rate,spo2,anchor_age,anchor_year,age,height,dod,los_hospital,admission_age,hospital_expire_flag,hospstay_seq,los_icu,icustay_seq,weight_admit,weight_daily,so2,po2,pco2,fio2_chartevents,fio2,aado2,aado2_calc,pao2fio2ratio,ph,baseexcess,totalco2,carboxyhemoglobin,methemoglobin,lactate,temperature,d_dimer,fibrinogen,thrombin,inr,pt,ptt,albumin,globulin,total_protein,aniongap,bun,creatinine,bicarbonate,calcium,chloride,glucose,potassium,sodium,mch,mchc,mcv,platelet,rbc,rdw,wbc,hematocrit,hemoglobin,urineoutput,gcs,gcs_motor,gcs_verbal,gcs_eyes,gcs_unable,alt,alp,ast,amylase,bilirubin_total,bilirubin_direct,bilirubin_indirect,ck_cpk,ck_mb,ggt,ld_ldh,vaso_rate_dobutamine,vaso_amount_dobutamine,vaso_rate_dopamine,vaso_amount_dopamine,vaso_rate_epinephrine,vaso_amount_epinephrine,vaso_rate_norepinephrine,vaso_amount_norepinephrine,vaso_rate_vasopressin,vaso_amount_vasopressin,dopamine,epinephrine,norepinephrine,phenylephrine,vasopressin,dobutamine,milrinone,pao2fio2ratio_novent,pao2fio2ratio_vent,rate_epinephrine,rate_norepinephrine,rate_dopamine,rate_dobutamine,meanbp_min,gcs_min,uo_24hr,bilirubin_max,creatinine_max,platelet_min,respiration_24hours,coagulation_24hours,liver_24hours,cardiovascular_24hours,cns_24hours,renal_24hours,sofa_24hours,antibiotic,positive_culture,suspected_infection,sepsis3,sofa_score,cardiovascular,cns,coagulation,liver,renal,respiration,shock_index,shock_index*age,so2fio2ratio
0,30004242,1,1,1,7,2180-11-19 00:00:00,157628.0,654840.0,69.0,165.0,73.0,96.0,165.0,73.0,65.0,27.5,98.0,75,2180,75,163.0,2152-10-30,7.579167,75,0,1,1.82,1,76.2,80.1,97.0,149.0,38.0,40.0,50.0,172.0,88.7,372.5,7.37,-2.0,23.0,0.0,0.0,1.4,36.78,9551.0,254.0,23.4,1.2,12.9,34.5,4.4,3.4,6.9,15.0,24.0,0.8,23.0,8.2,107.0,113.0,3.7,141.0,31.7,33.7,94.0,161.0,3.06,13.5,5.6,28.8,9.7,50.0,15.0,6.0,5.0,4.0,0.0,31.0,572.0,29.0,146.0,1.2,4.2,1.3,334.0,9.0,25.0,188.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.500884,0.01,0.030002,1.997283,2.4,0.500057,0.271739,268.571429,372.5,0.01,0.030002,2.500884,0.500057,96.0,15.0,1183.742591,1.2,0.8,161.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,3.0,1.0,2.0,0.0,0.0,0.0,0.0,0.418182,31.363636,1.94
1,30004242,1,1,1,7,2180-11-19 01:00:00,157628.0,654840.0,59.0,107.0,50.0,94.0,107.0,46.0,94.0,21.0,97.0,75,2180,75,163.0,2152-10-30,7.579167,75,0,1,1.82,1,76.2,80.1,97.0,149.0,38.0,40.0,50.0,172.0,88.7,372.5,7.37,-2.0,23.0,0.0,0.0,1.4,36.72,9551.0,254.0,23.4,1.2,12.9,34.5,4.4,3.4,6.9,15.0,24.0,0.8,23.0,8.2,107.0,113.0,3.7,141.0,31.7,33.7,94.0,161.0,3.06,13.5,5.6,28.8,9.7,50.0,15.0,6.0,5.0,4.0,0.0,31.0,572.0,29.0,146.0,1.2,4.2,1.3,334.0,9.0,25.0,188.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.500884,0.01,0.030002,1.997283,2.4,0.500057,0.271739,268.571429,372.5,0.01,0.030002,2.500884,0.500057,78.0,15.0,1183.742591,1.2,0.8,161.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,3.0,0.5,1.0,0.0,0.0,0.0,0.0,0.551402,41.355140,1.94
2,30004242,1,1,1,7,2180-11-19 02:00:00,157628.0,654840.0,60.0,92.0,74.0,78.0,92.0,74.0,78.0,19.0,97.0,75,2180,75,163.0,2152-10-30,7.579167,75,0,1,1.82,1,76.2,80.1,97.0,149.0,38.0,40.0,50.0,172.0,88.7,372.5,7.37,-2.0,23.0,0.0,0.0,1.4,36.72,9551.0,254.0,23.4,1.2,12.9,34.5,4.4,3.4,6.9,15.0,24.0,0.8,23.0,8.2,107.0,113.0,3.7,141.0,31.7,33.7,94.0,161.0,3.06,13.5,5.6,28.8,9.7,50.0,15.0,6.0,5.0,4.0,0.0,31.0,572.0,29.0,146.0,1.2,4.2,1.3,334.0,9.0,25.0,188.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.500884,0.01,0.030002,1.997283,2.4,0.500057,0.271739,268.571429,372.5,0.01,0.030002,2.500884,0.500057,56.0,15.0,1183.742591,1.2,0.8,161.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0,0.0,0.0,0,3.0,1.0,1.0,0.0,0.0,0.0,0.0,0.652174,48.913043,1.94
3,30004242,1,1,1,7,2180-11-19 03:00:00,157628.0,654840.0,46.0,132.0,47.0,74.5,120.0,34.0,56.0,16.0,96.0,75,2180,75,163.0,2152-10-30,7.579167,75,0,1,1.82,1,76.2,80.1,97.0,149.0,38.0,40.0,50.0,172.0,88.7,372.5,7.37,-2.0,23.

In [ ]:
columns = ['charttime'] + [col for col in df.columns if col != 'chartime']
df = df[columns]
df.head()

,charttime,stay_id,gender_encoded,first_hosp_stay_encoded,first_icu_stay_encoded,race_encoded,charttime,icu_stay_duration,hospital_stay_duration,heart_rate,sbp,dbp,mbp,sbp_ni,dbp_ni,mbp_ni,resp_rate,spo2,anchor_age,anchor_year,age,height,dod,los_hospital,admission_age,hospital_expire_flag,hospstay_seq,los_icu,icustay_seq,weight_admit,weight_daily,so2,po2,pco2,fio2_chartevents,fio2,aado2,aado2_calc,pao2fio2ratio,ph,baseexcess,totalco2,carboxyhemoglobin,methemoglobin,lactate,temperature,d_dimer,fibrinogen,thrombin,inr,pt,ptt,albumin,globulin,total_protein,aniongap,bun,creatinine,bicarbonate,calcium,chloride,glucose,potassium,sodium,mch,mchc,mcv,platelet,rbc,rdw,wbc,hematocrit,hemoglobin,urineoutput,gcs,gcs_motor,gcs_verbal,gcs_eyes,gcs_unable,alt,alp,ast,amylase,bilirubin_total,bilirubin_direct,bilirubin_indirect,ck_cpk,ck_mb,ggt,ld_ldh,vaso_rate_dobutamine,vaso_amount_dobutamine,vaso_rate_dopamine,vaso_amount_dopamine,vaso_rate_epinephrine,vaso_amount_epinephrine,vaso_rate_norepinephrine,vaso_amount_norepinephrine,vaso_rate_vasopressin,vaso_amount_vasopressin,dopamine,epinephrine,norepinephrine,phenylephrine,vasopressin,dobutamine,milrinone,pao2fio2ratio_novent,pao2fio2ratio_vent,rate_epinephrine,rate_norepinephrine,rate_dopamine,rate_dobutamine,meanbp_min,gcs_min,uo_24hr,bilirubin_max,creatinine_max,platelet_min,respiration_24hours,coagulation_24hours,liver_24hours,cardiovascular_24hours,cns_24hours,renal_24hours,sofa_24hours,antibiotic,positive_culture,suspected_infection,sepsis3,sofa_score,cardiovascular,cns,coagulation,liver,renal,respiration,shock_index,shock_index*age,so2fio2ratio
0,2180-11-19 00:00:00,30004242,1,1,1,7,2180-11-19 00:00:00,157628.0,654840.0,69.0,165.0,73.0,96.0,165.0,73.0,65.0,27.5,98.0,75,2180,75,163.0,2152-10-30,7.579167,75,0,1,1.82,1,76.2,80.1,97.0,149.0,38.0,40.0,50.0,172.0,88.7,372.5,7.37,-2.0,23.0,0.0,0.0,1.4,36.78,9551.0,254.0,23.4,1.2,12.9,34.5,4.4,3.4,6.9,15.0,24.0,0.8,23.0,8.2,107.0,113.0,3.7,141.0,31.7,33.7,94.0,161.0,3.06,13.5,5.6,28.8,9.7,50.0,15.0,6.0,5.0,4.0,0.0,31.0,572.0,29.0,146.0,1.2,4.2,1.3,334.0,9.0,25.0,188.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.500884,0.01,0.030002,1.997283,2.4,0.500057,0.271739,268.571429,372.5,0.01,0.030002,2.500884,0.500057,96.0,15.0,1183.742591,1.2,0.8,161.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,3.0,1.0,2.0,0.0,0.0,0.0,0.0,0.418182,31.363636,1.94
1,2180-11-19 01:00:00,30004242,1,1,1,7,2180-11-19 01:00:00,157628.0,654840.0,59.0,107.0,50.0,94.0,107.0,46.0,94.0,21.0,97.0,75,2180,75,163.0,2152-10-30,7.579167,75,0,1,1.82,1,76.2,80.1,97.0,149.0,38.0,40.0,50.0,172.0,88.7,372.5,7.37,-2.0,23.0,0.0,0.0,1.4,36.72,9551.0,254.0,23.4,1.2,12.9,34.5,4.4,3.4,6.9,15.0,24.0,0.8,23.0,8.2,107.0,113.0,3.7,141.0,31.7,33.7,94.0,161.0,3.06,13.5,5.6,28.8,9.7,50.0,15.0,6.0,5.0,4.0,0.0,31.0,572.0,29.0,146.0,1.2,4.2,1.3,334.0,9.0,25.0,188.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.500884,0.01,0.030002,1.997283,2.4,0.500057,0.271739,268.571429,372.5,0.01,0.030002,2.500884,0.500057,78.0,15.0,1183.742591,1.2,0.8,161.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,3.0,0.5,1.0,0.0,0.0,0.0,0.0,0.551402,41.355140,1.94
2,2180-11-19 02:00:00,30004242,1,1,1,7,2180-11-19 02:00:00,157628.0,654840.0,60.0,92.0,74.0,78.0,92.0,74.0,78.0,19.0,97.0,75,2180,75,163.0,2152-10-30,7.579167,75,0,1,1.82,1,76.2,80.1,97.0,149.0,38.0,40.0,50.0,172.0,88.7,372.5,7.37,-2.0,23.0,0.0,0.0,1.4,36.72,9551.0,254.0,23.4,1.2,12.9,34.5,4.4,3.4,6.9,15.0,24.0,0.8,23.0,8.2,107.0,113.0,3.7,141.0,31.7,33.7,94.0,161.0,3.06,13.5,5.6,28.8,9.7,50.0,15.0,6.0,5.0,4.0,0.0,31.0,572.0,29.0,146.0,1.2,4.2,1.3,334.0,9.0,25.0,188.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.500884,0.01,0.030002,1.997283,2.4,0.500057,0.271739,268.571429,372.5,0.01,0.030002,2.500884,0.500057,56.0,15.0,1183.742591,1.2,0.8,161.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0,0.0,0.0,0,3.0,1.0,1.0,0.0,0.0,0.0,0.0,0.652174,48.913043,1.94
3,2180-11-19 03:00:00,30004242,1,1,1,7,2180-11-19 03:00:00,157628.0,654840.0,46.0,132.0,47.0,74.5,120.0,34.0,56.0,16.0,96.0,75,2180,75,163.0,2152-10-30

In [ ]:
df = df.drop(columns=['dod','charttime'])
df.head()

,stay_id,gender_encoded,first_hosp_stay_encoded,first_icu_stay_encoded,race_encoded,icu_stay_duration,hospital_stay_duration,heart_rate,sbp,dbp,mbp,sbp_ni,dbp_ni,mbp_ni,resp_rate,spo2,anchor_age,anchor_year,age,height,los_hospital,admission_age,hospital_expire_flag,hospstay_seq,los_icu,icustay_seq,weight_admit,weight_daily,so2,po2,pco2,fio2_chartevents,fio2,aado2,aado2_calc,pao2fio2ratio,ph,baseexcess,totalco2,carboxyhemoglobin,methemoglobin,lactate,temperature,d_dimer,fibrinogen,thrombin,inr,pt,ptt,albumin,globulin,total_protein,aniongap,bun,creatinine,bicarbonate,calcium,chloride,glucose,potassium,sodium,mch,mchc,mcv,platelet,rbc,rdw,wbc,hematocrit,hemoglobin,urineoutput,gcs,gcs_motor,gcs_verbal,gcs_eyes,gcs_unable,alt,alp,ast,amylase,bilirubin_total,bilirubin_direct,bilirubin_indirect,ck_cpk,ck_mb,ggt,ld_ldh,vaso_rate_dobutamine,vaso_amount_dobutamine,vaso_rate_dopamine,vaso_amount_dopamine,vaso_rate_epinephrine,vaso_amount_epinephrine,vaso_rate_norepinephrine,vaso_amount_norepinephrine,vaso_rate_vasopressin,vaso_amount_vasopressin,dopamine,epinephrine,norepinephrine,phenylephrine,vasopressin,dobutamine,milrinone,pao2fio2ratio_novent,pao2fio2ratio_vent,rate_epinephrine,rate_norepinephrine,rate_dopamine,rate_dobutamine,meanbp_min,gcs_min,uo_24hr,bilirubin_max,creatinine_max,platelet_min,respiration_24hours,coagulation_24hours,liver_24hours,cardiovascular_24hours,cns_24hours,renal_24hours,sofa_24hours,antibiotic,positive_culture,suspected_infection,sepsis3,sofa_score,cardiovascular,cns,coagulation,liver,renal,respiration,shock_index,shock_index*age,so2fio2ratio
0,30004242,1,1,1,7,157628.0,654840.0,69.0,165.0,73.0,96.0,165.0,73.0,65.0,27.5,98.0,75,2180,75,163.0,7.579167,75,0,1,1.82,1,76.2,80.1,97.0,149.0,38.0,40.0,50.0,172.0,88.7,372.5,7.37,-2.0,23.0,0.0,0.0,1.4,36.78,9551.0,254.0,23.4,1.2,12.9,34.5,4.4,3.4,6.9,15.0,24.0,0.8,23.0,8.2,107.0,113.0,3.7,141.0,31.7,33.7,94.0,161.0,3.06,13.5,5.6,28.8,9.7,50.0,15.0,6.0,5.0,4.0,0.0,31.0,572.0,29.0,146.0,1.2,4.2,1.3,334.0,9.0,25.0,188.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.500884,0.01,0.030002,1.997283,2.4,0.500057,0.271739,268.571429,372.5,0.01,0.030002,2.500884,0.500057,96.0,15.0,1183.742591,1.2,0.8,161.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,3.0,1.0,2.0,0.0,0.0,0.0,0.0,0.418182,31.363636,1.94
1,30004242,1,1,1,7,157628.0,654840.0,59.0,107.0,50.0,94.0,107.0,46.0,94.0,21.0,97.0,75,2180,75,163.0,7.579167,75,0,1,1.82,1,76.2,80.1,97.0,149.0,38.0,40.0,50.0,172.0,88.7,372.5,7.37,-2.0,23.0,0.0,0.0,1.4,36.72,9551.0,254.0,23.4,1.2,12.9,34.5,4.4,3.4,6.9,15.0,24.0,0.8,23.0,8.2,107.0,113.0,3.7,141.0,31.7,33.7,94.0,161.0,3.06,13.5,5.6,28.8,9.7,50.0,15.0,6.0,5.0,4.0,0.0,31.0,572.0,29.0,146.0,1.2,4.2,1.3,334.0,9.0,25.0,188.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.500884,0.01,0.030002,1.997283,2.4,0.500057,0.271739,268.571429,372.5,0.01,0.030002,2.500884,0.500057,78.0,15.0,1183.742591,1.2,0.8,161.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,3.0,0.5,1.0,0.0,0.0,0.0,0.0,0.551402,41.355140,1.94
2,30004242,1,1,1,7,157628.0,654840.0,60.0,92.0,74.0,78.0,92.0,74.0,78.0,19.0,97.0,75,2180,75,163.0,7.579167,75,0,1,1.82,1,76.2,80.1,97.0,149.0,38.0,40.0,50.0,172.0,88.7,372.5,7.37,-2.0,23.0,0.0,0.0,1.4,36.72,9551.0,254.0,23.4,1.2,12.9,34.5,4.4,3.4,6.9,15.0,24.0,0.8,23.0,8.2,107.0,113.0,3.7,141.0,31.7,33.7,94.0,161.0,3.06,13.5,5.6,28.8,9.7,50.0,15.0,6.0,5.0,4.0,0.0,31.0,572.0,29.0,146.0,1.2,4.2,1.3,334.0,9.0,25.0,188.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.500884,0.01,0.030002,1.997283,2.4,0.500057,0.271739,268.571429,372.5,0.01,0.030002,2.500884,0.500057,56.0,15.0,1183.742591,1.2,0.8,161.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0,0.0,0.0,0,3.0,1.0,1.0,0.0,0.0,0.0,0.0,0.652174,48.913043,1.94
3,30004242,1,1,1,7,157628.0,654840.0,46.0,132.0,47.0,74.5,120.0,34.0,56.0,16.0,96.0,75,2180,75,163.0,7.579167,75,0,1,1.82,1,76.2,80.1,97.0,149.0,38.0,40.0,50.0,172.0,88.7,372.5,7.37,-2.0,23.0,0.0,0.0,1.4,36.72,9551.0,254.0,23.4,1.2,12.9,34.5,4.4,3.4,6.9,15.0,24.0,0.8,23.0,8.2,107.0,113.0,3.7,141.0,31.7,33.7,94.0,161.0,3.06,13.

In [ ]:
columns = [col for col in df.columns if col != 'los_icu'] + ['los_icu']
df = df[columns]
df.head()

,stay_id,gender_encoded,first_hosp_stay_encoded,first_icu_stay_encoded,race_encoded,icu_stay_duration,hospital_stay_duration,heart_rate,sbp,dbp,mbp,sbp_ni,dbp_ni,mbp_ni,resp_rate,spo2,anchor_age,anchor_year,age,height,los_hospital,admission_age,hospital_expire_flag,hospstay_seq,icustay_seq,weight_admit,weight_daily,so2,po2,pco2,fio2_chartevents,fio2,aado2,aado2_calc,pao2fio2ratio,ph,baseexcess,totalco2,carboxyhemoglobin,methemoglobin,lactate,temperature,d_dimer,fibrinogen,thrombin,inr,pt,ptt,albumin,globulin,total_protein,aniongap,bun,creatinine,bicarbonate,calcium,chloride,glucose,potassium,sodium,mch,mchc,mcv,platelet,rbc,rdw,wbc,hematocrit,hemoglobin,urineoutput,gcs,gcs_motor,gcs_verbal,gcs_eyes,gcs_unable,alt,alp,ast,amylase,bilirubin_total,bilirubin_direct,bilirubin_indirect,ck_cpk,ck_mb,ggt,ld_ldh,vaso_rate_dobutamine,vaso_amount_dobutamine,vaso_rate_dopamine,vaso_amount_dopamine,vaso_rate_epinephrine,vaso_amount_epinephrine,vaso_rate_norepinephrine,vaso_amount_norepinephrine,vaso_rate_vasopressin,vaso_amount_vasopressin,dopamine,epinephrine,norepinephrine,phenylephrine,vasopressin,dobutamine,milrinone,pao2fio2ratio_novent,pao2fio2ratio_vent,rate_epinephrine,rate_norepinephrine,rate_dopamine,rate_dobutamine,meanbp_min,gcs_min,uo_24hr,bilirubin_max,creatinine_max,platelet_min,respiration_24hours,coagulation_24hours,liver_24hours,cardiovascular_24hours,cns_24hours,renal_24hours,sofa_24hours,antibiotic,positive_culture,suspected_infection,sepsis3,sofa_score,cardiovascular,cns,coagulation,liver,renal,respiration,shock_index,shock_index*age,so2fio2ratio,los_icu
0,30004242,1,1,1,7,157628.0,654840.0,69.0,165.0,73.0,96.0,165.0,73.0,65.0,27.5,98.0,75,2180,75,163.0,7.579167,75,0,1,1,76.2,80.1,97.0,149.0,38.0,40.0,50.0,172.0,88.7,372.5,7.37,-2.0,23.0,0.0,0.0,1.4,36.78,9551.0,254.0,23.4,1.2,12.9,34.5,4.4,3.4,6.9,15.0,24.0,0.8,23.0,8.2,107.0,113.0,3.7,141.0,31.7,33.7,94.0,161.0,3.06,13.5,5.6,28.8,9.7,50.0,15.0,6.0,5.0,4.0,0.0,31.0,572.0,29.0,146.0,1.2,4.2,1.3,334.0,9.0,25.0,188.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.500884,0.01,0.030002,1.997283,2.4,0.500057,0.271739,268.571429,372.5,0.01,0.030002,2.500884,0.500057,96.0,15.0,1183.742591,1.2,0.8,161.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,3.0,1.0,2.0,0.0,0.0,0.0,0.0,0.418182,31.363636,1.94,1.82
1,30004242,1,1,1,7,157628.0,654840.0,59.0,107.0,50.0,94.0,107.0,46.0,94.0,21.0,97.0,75,2180,75,163.0,7.579167,75,0,1,1,76.2,80.1,97.0,149.0,38.0,40.0,50.0,172.0,88.7,372.5,7.37,-2.0,23.0,0.0,0.0,1.4,36.72,9551.0,254.0,23.4,1.2,12.9,34.5,4.4,3.4,6.9,15.0,24.0,0.8,23.0,8.2,107.0,113.0,3.7,141.0,31.7,33.7,94.0,161.0,3.06,13.5,5.6,28.8,9.7,50.0,15.0,6.0,5.0,4.0,0.0,31.0,572.0,29.0,146.0,1.2,4.2,1.3,334.0,9.0,25.0,188.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.500884,0.01,0.030002,1.997283,2.4,0.500057,0.271739,268.571429,372.5,0.01,0.030002,2.500884,0.500057,78.0,15.0,1183.742591,1.2,0.8,161.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,3.0,0.5,1.0,0.0,0.0,0.0,0.0,0.551402,41.355140,1.94,1.82
2,30004242,1,1,1,7,157628.0,654840.0,60.0,92.0,74.0,78.0,92.0,74.0,78.0,19.0,97.0,75,2180,75,163.0,7.579167,75,0,1,1,76.2,80.1,97.0,149.0,38.0,40.0,50.0,172.0,88.7,372.5,7.37,-2.0,23.0,0.0,0.0,1.4,36.72,9551.0,254.0,23.4,1.2,12.9,34.5,4.4,3.4,6.9,15.0,24.0,0.8,23.0,8.2,107.0,113.0,3.7,141.0,31.7,33.7,94.0,161.0,3.06,13.5,5.6,28.8,9.7,50.0,15.0,6.0,5.0,4.0,0.0,31.0,572.0,29.0,146.0,1.2,4.2,1.3,334.0,9.0,25.0,188.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.500884,0.01,0.030002,1.997283,2.4,0.500057,0.271739,268.571429,372.5,0.01,0.030002,2.500884,0.500057,56.0,15.0,1183.742591,1.2,0.8,161.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0,0.0,0.0,0,3.0,1.0,1.0,0.0,0.0,0.0,0.0,0.652174,48.913043,1.94,1.82
3,30004242,1,1,1,7,157628.0,654840.0,46.0,132.0,47.0,74.5,120.0,34.0,56.0,16.0,96.0,75,2180,75,163.0,7.579167,75,0,1,1,76.2,80.1,97.0,149.0,38.0,40.0,50.0,172.0,88.7,372.5,7.37,-2.0,23.0,0.0,0.0,1.4,36.72,9551.0,254.0,23.4,1.2,12.9,34.5,4.4,3.4,6.9,15.0,24.0,0.8,23.0,8.2,107.0,113.0,3.7,141.0,31.7,33.7,94.0,161.0,3.06,13.5,5.6

In [ ]:
!pip install xgboost
!pip install scikit-optimize

In [ ]:
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler

# Assuming df is your original DataFrame with all features including 'stay_id' and the target column 'los_icu'
# Exclude 'stay_id' from the features used for Lasso
X = df.drop(['los_icu', 'stay_id'], axis=1)  # Drop target and stay_id columns
y = df['los_icu']

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Fit the Lasso model with a smaller alpha
lasso = Lasso(alpha=0.0001)  # Adjust alpha based on your needs
lasso.fit(X_scaled, y)

# Select the non-zero coefficients
model = SelectFromModel(lasso, prefit=True)
X_selected = model.transform(X_scaled)

# Get the columns that were selected
selected_columns = X.columns[model.get_support()]

# Check if any features were selected
if len(selected_columns) > 0:
    # Create a new DataFrame with the selected columns, the stay_id, and the target
    df_selected = pd.DataFrame(X_selected, columns=selected_columns)
    df_selected['stay_id'] = df['stay_id'].values  # Add the stay_id column back
    df_selected['los_icu'] = y  # Add the target column back
    df=df_selected

else:
    print("No features were selected by Lasso.")
    df_selected = df[['stay_id', 'los_icu']]  # Create a DataFrame with only the stay_id and target column
    df=df_selected
# Display the new DataFrame
print(df.head())




   first_icu_stay_encoded  icu_stay_duration       dbp  resp_rate  \
0                0.200222          -0.801409  0.705471   1.510330   
1                0.200222          -0.801409 -0.996072   0.236029   
2                0.200222          -0.801409  0.779451  -0.156064   
3                0.200222          -0.801409 -1.218012  -0.744203   
4                0.200222          -0.801409 -0.848111  -0.352110   

   anchor_year  los_hospital  icustay_seq  weight_admit  weight_daily  \
0     1.254766     -0.605573    -0.193655     -0.305859     -0.288385   
1     1.254766     -0.605573    -0.193655     -0.305859     -0.288385   
2     1.254766     -0.605573    -0.193655     -0.305859     -0.288385   
3     1.254766     -0.605573    -0.193655     -0.305859     -0.288385   
4     1.254766     -0.605573    -0.193655     -0.305859     -0.288385   

      fio2  fibrinogen       ptt  globulin  bicarbonate      mch      mchc  \
0 -0.10207   -0.326479 -0.087892  1.275093    -0.318382  0.66582  0.

In [ ]:
print(df.shape[1])

31


In [ ]:
#PAVLOS
from skopt import BayesSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor
class Models:

    def __init__(self):
        pass

    '''Linear Regression Model''' # Aggelos
    def LinearRegressor(self, X_train, X_test, y_train):
        from sklearn.linear_model import LinearRegression
        regressor = LinearRegression() # Create a Linear Regression model
        regressor.fit(X_train, y_train) # Train the model using the training data

        # Make predictions on the test set
        y_pred = regressor.predict(X_test)
        return y_pred # Return the predicted values



    '''Polynomial Regression Model''' # Pavlos
    def PolynomialRegressor(self, X_train, X_test, y_train):
        from sklearn.decomposition import PCA
        pca = PCA(n_components=30) # Reduce dimensionality to 20 principal components
        X_train_reduced = pca.fit_transform(X_train) # Fit and transform the training data
        X_test_reduced = pca.transform(X_test) # Transform the test data using the same PCA

        from sklearn.preprocessing import PolynomialFeatures
        poly_reg = PolynomialFeatures(degree=2) # Generate polynomial features of degree 3
        X_poly_train = poly_reg.fit_transform(X_train_reduced) # Fit and transform the reduced training data
        X_poly_test = poly_reg.transform(X_test_reduced) # Transform the reduced test data

        from sklearn.linear_model import LinearRegression
        lin_reg_2 = LinearRegression() # Create a Linear Regression model
        lin_reg_2.fit(X_poly_train, y_train) # Train the model on the polynomial features

        y_pred = lin_reg_2.predict(X_poly_test) # Make predictions on the polynomial test data
        return y_pred # Return the predicted values



    '''SVM Regression Model''' # Pavlos
    def SVMRegressor(self, X_train, X_test, y_train):
        from sklearn.decomposition import PCA
        pca = PCA(n_components=20)  # Reduce dimensionality to 20 principal components
        X_train_reduced = pca.fit_transform(X_train) # Fit and transform the training data
        X_test_reduced = pca.transform(X_test) # Transform the test data using the same PCA

        from sklearn.preprocessing import StandardScaler
        sc_X = StandardScaler() # Create a StandardScaler for features
        sc_y = StandardScaler() # Create a StandardScaler for the target variable
        X = sc_X.fit_transform( X_train_reduced) # Fit and scale the reduced training data
        y = sc_y.fit_transform(y_train.reshape(-1, 1)).flatten() # Fit and scale the target variable, then flatten

        from sklearn.svm import SVR
        regressor = SVR(kernel='rbf') # Create an SVR model with an RBF kernel
        regressor.fit(X, y) # Train the model using the training data

        # Make predictions on the test set
        y_pred_scaled = regressor.predict(sc_X.transform(X_test_reduced))
        # Inverse transform the predicted values to the original scale
        y_pred = sc_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()
        return y_pred # Return the predicted values



    '''Decision Tree Regression Model''' # Michail
    def DecisionTreeRegressor(self, X_train, X_test, y_train):
        from sklearn.tree import DecisionTreeRegressor
        regressor = DecisionTreeRegressor(criterion='poisson') # Create a Decision Tree Regressor model
        regressor.fit(X_train,y_train) # Train the model using the training data

        # Make predictions on the test set
        y_pred = regressor.predict(X_test)
        return y_pred # Return the predicted values



    '''Random Forest Regression Model''' # Aggelos
    def RandomForestRegressor(self, X_train, X_test, y_train):
      # Define the parameter search space
      search_space = {
          'n_estimators': (10, 200),              # Number of trees in the forest
          'max_depth': (1, 30),                   # Maximum depth of the trees
          'min_samples_split': (2, 20),           # Minimum number of samples required to split an internal node
          'min_samples_leaf': (1, 20),            # Minimum number of samples required to be at a leaf node
          'max_features': ['sqrt', 'log2', None]  # Number of features to consider when looking for the best split
      }

      # Initialize the Bayesian optimizer
      opt = BayesSearchCV(
          estimator=RandomForestRegressor(),
          search_spaces=search_space,
          n_iter=20,  # Number of iterations
          cv=15,       # Number of cross-validation folds
          scoring='neg_mean_squared_error',  # Use negative MSE as the scoring metric
          n_jobs=-1,  # Use all available CPUs
          verbose=1
      )

      # Fit the optimizer
      opt.fit(X_train, y_train)

      # Best model
      best_model = opt.best_estimator_

      # Predict on the test set using the best model
      y_pred = best_model.predict(X_test)

      # Evaluate performance on the test set
      mse = mean_squared_error(y_test, y_pred)
      print(f"Best parameters: {opt.best_params_}")
      print(f"Mean Squared Error: {mse}")

      return y_pred, opt.best_params_



    '''XGboost Regression Model''' # Pavlos
    def XGboostRegressor(self,X_train, y_train, X_test):
    # Define the parameter search space
      search_space = {
        'n_estimators': (50, 500),             # Number of boosting rounds
        'max_depth': (3, 10),                  # Maximum tree depth for base learners
        'learning_rate': (0.01, 0.3, 'log-uniform'),  # Step size shrinkage
        'subsample': (0.5, 1.0),               # Subsample ratio of the training instances
        'colsample_bytree': (0.5, 1.0)         # Subsample ratio of columns when constructing each tree
    }

    # Initialize the Bayesian optimizer
      opt = BayesSearchCV(
        estimator=xgb.XGBRegressor(),
        search_spaces=search_space,
        n_iter=20,  # Number of iterations (adjust as needed)
        cv=15,       # Number of cross-validation folds
        scoring='neg_mean_squared_error',  # Use negative MSE as the scoring metric
        n_jobs=-1,  # Use all available CPUs
        verbose=1
    )

    # Fit the optimizer
      opt.fit(X_train, y_train)

    # Best model
      best_model = opt.best_estimator_

    # Predict on the test set using the best model
      y_pred = best_model.predict(X_test)

    # Evaluate performance on the test set
      mse = mean_squared_error(y_test, y_pred)
      print(f"Best parameters: {opt.best_params_}")
      print(f"Mean Squared Error: {mse}")

      return y_pred, opt.best_params_




    def LightGBMRegressor(self,X_train, y_train, X_test):
    # Define the parameter search space
      search_space = {
        'n_estimators': (50, 1000),              # Number of boosting rounds
        'max_depth': (3, 12),                    # Maximum tree depth for base learners
        'learning_rate': (0.01, 0.3, 'log-uniform'),  # Step size shrinkage
        'num_leaves': (20, 100),                 # Number of leaves in one tree
        'min_child_samples': (5, 100),           # Minimum number of data points required in a leaf
        'subsample': (0.5, 1.0),                 # Subsample ratio of the training instances
        'colsample_bytree': (0.5, 1.0)           # Subsample ratio of columns when constructing each tree
    }

    # Initialize the Bayesian optimizer
      opt = BayesSearchCV(
        estimator=LGBMRegressor(),
        search_spaces=search_space,
        n_iter=15,  # Number of iterations (adjust as needed)
        cv=10,       # Number of cross-validation folds
        scoring='neg_mean_squared_error',  # Use negative MSE as the scoring metric
        n_jobs=-1,  # Use all available CPUs
        verbose=1
    )

    # Fit the optimizer
      opt.fit(X_train, y_train)

    # Best model
      best_model = opt.best_estimator_

    # Predict on the test set using the best model
      y_pred = best_model.predict(X_test)

    # Evaluate performance on the test set
      mse = mean_squared_error(y_test, y_pred)
      print(f"Best parameters: {opt.best_params_}")
      print(f"Mean Squared Error: {mse}")

      return y_pred, opt.best_params_



    '''ANN Regression Model''' # Pavlos
    def ANNRegressor(self, X_train, X_test, y_train):
        from sklearn.preprocessing import StandardScaler
        sc = StandardScaler() # Create a StandardScaler
        X_train = sc.fit_transform(X_train) # Fit and scale the training data
        X_test = sc.transform(X_test) # Scale the test data using the same scaler

        # Import TensorFlow and create a Sequential model for the neural network (ANN)
        import tensorflow as tf
        ann = tf.keras.models.Sequential()

        ann.add(tf.keras.layers.Dense(units=6, activation='relu')) # Add the input layer and the first hidden layer with 6 neurons and ReLU activation function
        ann.add(tf.keras.layers.Dense(units=6, activation='relu')) # Add the second hidden layer with 6 neurons and ReLU activation function
        ann.add(tf.keras.layers.Dense(units=1, activation='linear')) # Add the output layer with 1 neuron and linear activation function (for continues values predictions)

        # Compile the ANN with Adam optimizer, mean squared error loss function, and mean squared error as a metric
        ann.compile(optimizer='adam' , loss='mean_squared_error' , metrics=['mean_squared_error'] )

        # Train the ANN on the training data with a batch size of 32 and for 10 epochs
        ann.fit(X_train, y_train, batch_size=32, epochs=10)

        # Predict the test set results
        y_pred = ann.predict(X_test)
        return y_pred



    # The evaluation function
    def Evaluations(self, y_test,y_pred):
        import numpy as np
        # Import evaluate metricks for the model
        from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
        # Calculate evaluation metrics
        mae = mean_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        r2 = r2_score(y_test, y_pred)


        # Print the evaluation metrics
        print('Mean Absolute Error (MAE):', mae)
        print('Mean Squared Error (MSE):', mse)
        print('Root Mean Squared Error (RMSE):', rmse)
        print('R-squared (R²):', r2)

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
#import pandas as pd
#from sklearn.preprocessing import StandardScaler


#df_first_col = df.iloc[:, [0]]
#df_to_scale = df.iloc[:, 1:]

#scaler = StandardScaler()
#scaled_columns = scaler.fit_transform(df_to_scale)

#df_scaled = pd.DataFrame(scaled_columns, columns=df_to_scale.columns)

#df = pd.concat([df_first_col, df_scaled], axis=1)
#file_path = '/content/gdrive/MyDrive/ICU_LOSS_SCALED'

# Save DataFrame to the specified CSV file
#df.to_csv(file_path, index=False)

In [ ]:
#Pavlos
# Import library
import pandas as pd
from sklearn.model_selection import train_test_split  # Ensure this import is included
#from models import Models  # A made module in the file 'models.py'

# Load the dataset from a CSV file
#df = pd.read_csv('icu_loss.csv')

# Extract the unique patient IDs
unique_patients = df['stay_id'].unique()

# Split the unique patient IDs into training and testing sets
train_patients, test_patients = train_test_split(unique_patients, test_size=0.2, random_state=42)

# Create training and testing datasets based on patient IDs
df_train = df[df['stay_id'].isin(train_patients)]
df_test = df[df['stay_id'].isin(test_patients)]



# Separate features and target for both training and testing sets
X_train = df_train.drop(columns=['los_icu']).to_numpy()
y_train = df_train['los_icu'].to_numpy()
X_test = df_test.drop(columns=['los_icu']).to_numpy()
y_test = df_test['los_icu'].to_numpy()


# Create an instance of the models class
model = Models()

'''Linear Regression'''
y_pred = model.LinearRegressor(X_train, X_test, y_train)

# Print evaluation results
print("Linear Regression Model Evaluations")
model.Evaluations(y_test, y_pred)



'''Polynomial Regression'''
y_pred = model.PolynomialRegressor(X_train, X_test, y_train)

# Print evaluation results
print("Polynomial Regression Model Evaluations")
model.Evaluations(y_test, y_pred)



'''SVM Regression'''
y_pred = model.SVMRegressor(X_train, X_test, y_train)

# Print evaluation results
print("SVM Regression Model Evaluations")
model.Evaluations(y_test, y_pred)



'''Decision Tree Regression Model'''
y_pred = model.DecisionTreeRegressor(X_train, X_test, y_train)

# Print evaluation results
print("Decision Tree Regression Model Evaluations")
model.Evaluations(y_test, y_pred)



'''Random Forest Regression Model'''
y_pred, best_params = model.RandomForestRegressor(X_train, X_test, y_train)

# Print evaluation results
print("Random Forest Regression Model Evaluations")
model.Evaluations(y_test, y_pred)
print(best_params)



'''XGboost Regression Model'''
y_pred,best_params = model.XGboostRegressor(X_train, y_train,X_test)

# Print evaluation results
print("XGboost Regression Model Evaluations")
model.Evaluations(y_test, y_pred)



'''LightGBM Regression Model'''
y_pred, _ = model.LightGBMRegressor(X_train, y_train, X_test) # Modified line
# Print evaluation results
print("LightGBM Regression Model Evaluations")
model.Evaluations(y_test, y_pred)


'''Gradient Boost Regression Model'''
y_pred = model.GradientBoostRegressor(X_train, X_test, y_train)

# Print evaluation results
print("Gradient Boost Regression Model Evaluations")
model.Evaluations(y_test, y_pred)



'''ANN Regression Model'''
y_pred = model.ANNRegressor(X_train, X_test, y_train)

# Print evaluation results
print("ANN Regression Model Evaluations")
model.Evaluations(y_test, y_pred)



Fitting 15 folds for each of 1 candidates, totalling 15 fits
Fitting 15 folds for each of 1 candidates, totalling 15 fits
Fitting 15 folds for each of 1 candidates, totalling 15 fits
Fitting 15 folds for each of 1 candidates, totalling 15 fits
Fitting 15 folds for each of 1 candidates, totalling 15 fits


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Fitting 15 folds for each of 1 candidates, totalling 15 fits
Fitting 15 folds for each of 1 candidates, totalling 15 fits
Fitting 15 folds for each of 1 candidates, totalling 15 fits
Fitting 15 folds for each of 1 candidates, totalling 15 fits
Fitting 15 folds for each of 1 candidates, totalling 15 fits
Fitting 15 folds for each of 1 candidates, totalling 15 fits
Fitting 15 folds for each of 1 candidates, totalling 15 fits


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Fitting 15 folds for each of 1 candidates, totalling 15 fits
Fitting 15 folds for each of 1 candidates, totalling 15 fits
Fitting 15 folds for each of 1 candidates, totalling 15 fits
Fitting 15 folds for each of 1 candidates, totalling 15 fits
Fitting 15 folds for each of 1 candidates, totalling 15 fits
Fitting 15 folds for each of 1 candidates, totalling 15 fits
Fitting 15 folds for each of 1 candidates, totalling 15 fits
Fitting 15 folds for each of 1 candidates, totalling 15 fits
Best parameters: OrderedDict([('colsample_bytree', 0.9144255960027838), ('learning_rate', 0.22670213629618166), ('max_depth', 3), ('n_estimators', 50), ('subsample', 1.0)])
Mean Squared Error: 1.2885653465905127
XGboost Regression Model Evaluations
Mean Absolute Error (MAE): 0.36966332226757453
Mean Squared Error (MSE): 1.2885653465905127
Root Mean Squared Error (RMSE): 1.1351499225170711
R-squared (R²): 0.9857004436909329


'\n\n\'\'\'ANN Regression Model\'\'\'\ny_pred = model.ANNRegressor(X_train, X_test, y_train)\n\n# Print evaluation results\nprint("ANN Regression Model Evaluations")\nmodel.Evaluations(y_test, y_pred)\n'